In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pytz
import ta

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime
import MetaTrader5 as mt5

from ortisan_ta.dataaccess import DataItem, MetaTraderDataAccess
import ortisan_ta.utils.analysis as ortisan_ta

from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volume import OnBalanceVolumeIndicator
from ortisan_ta.simulator import MarketSimulator


In [2]:
#Settings
mpl.style.use('seaborn')
default_fig_size = (20, 20) # Width , Height

### Symbols in my historical database

In [3]:
symbols = ['AALC34',
'AAPL11B',
'ABCB11',
'ABEV3',
'ABRE3',
'ABTT11B',
'ABYA3',
'ACES4',
'ACNB34',
'AEDU3',
'AELP3',
'AETB34',
'AFLT3',
'AFLU3',
'AGEI3',
'ALLL4',
'ALPA3',
'ALUP4',
'AMBV4',
'AMCE3',
'AMGN34',
'AMIL3',
'AMPI3',
'AMZO34',
'ANIM3',
'APHP3B',
'APTV34',
'ARCE3',
'ARLA4',
'ARNC34',
'ARZZ3',
'ASML34',
'ATVI34',
'AVGO34',
'AVON11B',
'AVPL3',
'BABA34',
'BAHI4',
'BALM3',
'BALM4',
'BBRK3',
'BBSD11',
'BBTG12',
'BBTG35',
'BBTG36',
'BEES3',
'BEES4',
'BESP3',
'BESP4',
'BICB3',
'BIDI11',
'BIDU34',
'BIIB34',
'BIOM3',
'BIOM4',
'BKBR3',
'BKNG34',
'BMEB3',
'BMGB11',
'BMIN3',
'BMYB34',
'BNCA3',
'BOAC11B',
'BOVA11',
'BOVH3',
'BOVV11',
'BPAC3',
'BPAN4',
'BPHA3',
'BRAP3',
'BRFS3',
'BRGE7',
'BRGE8',
'BRIV4',
'BRKM6',
'BRML3',
'BRSR3',
'BRTO3',
'BRTO4',
'BSAN33',
'BSCT5',
'BSEV3',
'BTOW3',
'BUET4',
'CAFE3',
'CAJI34',
'CALI4',
'CAMB3',
'CAPH34',
'CASN3',
'CBAG3',
'CCHI4',
'CCPR3',
'CCXC3',
'CEBR3',
'CEEB3',
'CEEB5',
'CEED4B',
'CEGR3',
'CELP3',
'CELP5',
'CELP6',
'CEPE5',
'CESP6',
'CGAS11',
'CGRA4',
'CLAN4',
'CLSC5',
'CLSC6',
'CMGR4',
'CMIG4',
'CNFB3',
'CNFB4',
'COBE3B',
'COBE5B',
'COCA11B',
'COCA34',
'COCE3',
'COPH34',
'COWC34',
'CPLE3',
'CPNY3',
'CPSL3',
'CRBM3',
'CRBM7',
'CRHP34',
'CRIV3',
'CRPG6',
'CRUZ3',
'CSAB4',
'CSCO11B',
'CSMG12',
'CSMG3',
'CSRN3',
'CSRN6',
'CTIP3',
'CTKA3',
'CTKA4',
'CTNM3',
'CTNM4',
'CTPC3',
'CTPC4',
'CVCB3',
'DAGB33',
'DASA3',
'DBAG34',
'DEAI34',
'DELL11B',
'DFVA3',
'DFVA4',
'DHBI4',
'DIRR3',
'DISB11B',
'DIVO11',
'DLTR34',
'DOCA4',
'DPPI3',
'DTEX3',
'DUPO34',
'DUQE3',
'DURA3',
'DURA4',
'DWDP34',
'DXTG4',
'EAIN34',
'EALT3',
'EALT4',
'EBAY11B',
'ECOR3',
'EEEL4',
'EEEL4B',
'EGIE3',
'ELEK3',
'ELET6',
'ELPL3',
'EMBR3',
'ENGI3',
'EQPA5',
'ESUD11',
'EXGR34',
'FBMC3',
'FBMC4',
'FCAP3',
'FCAP4',
'FCXO34',
'FDMO34',
'FDXB34',
'FESA4',
'FFTL4',
'FGUI3',
'FIBR3',
'FIGE4',
'FLRY3',
'FMSC34',
'FNAM11',
'FRAS4',
'FRIO3',
'FSTU11',
'GAZO3',
'GAZO4',
'GBIO33',
'GDBR34',
'GEOO11B',
'GEOO34',
'GETI3',
'GETI4',
'GGBR4',
'GILD34',
'GLOB3',
'GLOB4',
'GMCO34',
'GNDI3',
'GOAU3',
'GOAU4',
'GOOG35',
'GPAR3',
'GPIV33',
'GPRO34',
'GPSI34',
'GRNL3',
'GSGI11B',
'GSGI34',
'GSHP3',
'GTDP4B',
'GUAR3',
'HAGA3',
'HAGA4',
'HALI11B',
'HALI34',
'HBOR3',
'HGTX3',
'HOME34',
'HONB34',
'HRTP3',
'IBMB34',
'IBOV11',
'IDVL3',
'IENG3',
'IGBR7',
'IGTA3',
'ILMD3',
'ILMD4',
'IMBI3',
'IMCH3',
'INPR3',
'ISBC34',
'ISUS11',
'ITLC11B',
'ITLC34',
'IVTL11B',
'JBDU11',
'JBDU12',
'JBSS3',
'JCPC34',
'JDCO34',
'JFEN3',
'JHSF3',
'JNJB11B',
'JNJB34',
'JPSA4',
'JSLG3',
'KHCB34',
'KMBB34',
'KROT3',
'KROT4',
'KSSA3',
'LAME4',
'LARK4',
'LETO5',
'LEVE4',
'LFFE3',
'LILY34',
'LIPR3',
'LIQO3',
'LIXC3',
'LLIS3',
'LNKD11B',
'LOGG3',
'LPSB3',
'LREN3',
'LUXM4',
'MACY34',
'MAGS3',
'MAOR3B',
'MAOR4B',
'MARI3',
'MATB11',
'MDIA3',
'MDTC34',
'MEDI3',
'MERC4',
'MGEL4',
'MILK33',
'MILS3',
'MKLC34',
'MLPA12',
'MLPA4',
'MMMC34',
'MNDL4',
'MNPR3',
'MOVI3',
'MRCK34',
'MRSA5B',
'MRSA6B',
'MSBR11B',
'MSBR34',
'MSFT34',
'MSPA4',
'MSTO34',
'MTIG3',
'MTSA3',
'MWET3',
'NAFG3',
'NEOE3',
'NETC4',
'NIKE34',
'NMRH34',
'NOKI34',
'NUTR3',
'NVDC34',
'ODPV3',
'OGSA3',
'OGXP3',
'OHLB3',
'OMGE3',
'OXYP34',
'PCAR3',
'PCAR4',
'PCAR5',
'PDGR3',
'PEAB3',
'PEFX3',
'PEPB34',
'PETR4',
'PFRM3',
'PGCO11B',
'PHGN34',
'PIBB11',
'PLTO5',
'PMAM3',
'PMAM4',
'PNVL3',
'PNVL4',
'POMO3',
'POMO4',
'POSI3',
'PPLA11',
'PPLA35',
'PREB11',
'PRVI3',
'PTBL4',
'PTCH34',
'PTIP3',
'PTIP4',
'PTNT3',
'PTPA4',
'PYPL34',
'QCOM34',
'QGEP3',
'RADL3',
'RAIA3',
'RAIL3',
'RDNI3',
'REDE3',
'REDE4',
'REEM4',
'RHDS4',
'RNEW3',
'RNPT4',
'ROST34',
'RPAD3',
'RPAD6',
'RPMG3',
'RPMG4',
'RSIP3',
'RUMO3',
'SANB11',
'SANB3',
'SANC34',
'SAPP34',
'SAPR3',
'SASG3',
'SATI3',
'SBNY34',
'SBSP3',
'SBUB34',
'SCLO3',
'SEER3',
'SGAS3',
'SHUL3',
'SJOS4',
'SLBC34',
'SLBG11B',
'SLBG34',
'SLCE3',
'SLED3',
'SLED4',
'SMAL11',
'SMTO3',
'SNEC34',
'SNSL3',
'SPRI3',
'SPRI6',
'SPRT3B',
'SPXI11',
'SRXM34',
'SSBR3',
'SSFO34',
'STBP3',
'STBR11',
'SUBA3',
'SULT4',
'SUZB3',
'SUZB5',
'SUZB6',
'SZPQ4',
'TAEE4',
'TAKP34',
'TAMM3',
'TAMM4',
'TASA3',
'TBLE3',
'TCSL3',
'TEFC11',
'TEKA3',
'TELB3',
'TEMP3',
'TENE7',
'TERI3',
'TEXA34',
'TGLT11',
'TGMA3',
'TGTB34',
'TIET3',
'TIET4',
'TKNO3',
'TKNO4',
'TLPP3',
'TLPP4',
'TMAC11B',
'TMAC3B',
'TMAC5B',
'TMCP4',
'TMGC11',
'TMGC12',
'TMGC3',
'TMGC7',
'TNCP4',
'TOTS3',
'TOYB4',
'TPRY34',
'TRIS3',
'TRNA4',
'TROR3',
'TSLA34',
'TSMC34',
'TUPY3',
'TUPY4',
'TWXB34',
'TXRX3',
'TXSA34',
'U1BE34',
'UBBR11',
'UBBR3',
'UGPA3',
'ULEV34',
'UNIP5',
'UPAC34',
'UPSS34',
'USBC34',
'USIM5',
'USIM6',
'VAGV3',
'VAGV4',
'VERZ34',
'VFCO34',
'VGOR4',
'VIGR3',
'VINE5',
'VINE6',
'VIVO12',
'VIVO3',
'VIVO4',
'VIVT3',
'VNET3',
'VPTA3',
'VPTA4',
'VSPT3',
'VTLM3',
'VULC4',
'VVAR3',
'WALM11B',
'WFCO11B',
'WFCO34',
'WIZS3',
'WMBY3',
'WSON33',
'XPOM11',
'YDUQ3',
'YUMR34']


In [4]:
symbols

['AALC34',
 'AAPL11B',
 'ABCB11',
 'ABEV3',
 'ABRE3',
 'ABTT11B',
 'ABYA3',
 'ACES4',
 'ACNB34',
 'AEDU3',
 'AELP3',
 'AETB34',
 'AFLT3',
 'AFLU3',
 'AGEI3',
 'ALLL4',
 'ALPA3',
 'ALUP4',
 'AMBV4',
 'AMCE3',
 'AMGN34',
 'AMIL3',
 'AMPI3',
 'AMZO34',
 'ANIM3',
 'APHP3B',
 'APTV34',
 'ARCE3',
 'ARLA4',
 'ARNC34',
 'ARZZ3',
 'ASML34',
 'ATVI34',
 'AVGO34',
 'AVON11B',
 'AVPL3',
 'BABA34',
 'BAHI4',
 'BALM3',
 'BALM4',
 'BBRK3',
 'BBSD11',
 'BBTG12',
 'BBTG35',
 'BBTG36',
 'BEES3',
 'BEES4',
 'BESP3',
 'BESP4',
 'BICB3',
 'BIDI11',
 'BIDU34',
 'BIIB34',
 'BIOM3',
 'BIOM4',
 'BKBR3',
 'BKNG34',
 'BMEB3',
 'BMGB11',
 'BMIN3',
 'BMYB34',
 'BNCA3',
 'BOAC11B',
 'BOVA11',
 'BOVH3',
 'BOVV11',
 'BPAC3',
 'BPAN4',
 'BPHA3',
 'BRAP3',
 'BRFS3',
 'BRGE7',
 'BRGE8',
 'BRIV4',
 'BRKM6',
 'BRML3',
 'BRSR3',
 'BRTO3',
 'BRTO4',
 'BSAN33',
 'BSCT5',
 'BSEV3',
 'BTOW3',
 'BUET4',
 'CAFE3',
 'CAJI34',
 'CALI4',
 'CAMB3',
 'CAPH34',
 'CASN3',
 'CBAG3',
 'CCHI4',
 'CCPR3',
 'CCXC3',
 'CEBR3',
 'CEEB3',
 'CE

### Gets the data for each symbol

In [5]:
data_access = MetaTraderDataAccess()
dfs = data_access.get_rates_from_symbols(symbols, datetime(2020, 1, 1), datetime(2020, 12, 4), mt5.TIMEFRAME_H1)
dfs

{'AALC34': Empty DataFrame
 Columns: []
 Index: [],
 'AAPL11B': Empty DataFrame
 Columns: []
 Index: [],
 'ABCB11': Empty DataFrame
 Columns: []
 Index: [],
 'ABEV3':                       Open   High    Low  Close  Spread   Volume
 Date                                                            
 2020-01-02 07:00:00  18.86  18.96  18.78  18.94       1  1503500
 2020-01-02 08:00:00  18.94  19.16  18.91  19.16       1  3092900
 2020-01-02 09:00:00  19.16  19.22  19.13  19.19       1  1742000
 2020-01-02 10:00:00  19.18  19.25  19.17  19.18       1  1865600
 2020-01-02 11:00:00  19.17  19.20  19.15  19.16       1   991200
 ...                    ...    ...    ...    ...     ...      ...
 2020-12-03 10:00:00  14.64  14.74  14.59  14.66       1  3917200
 2020-12-03 11:00:00  14.64  14.75  14.62  14.65       1  3184300
 2020-12-03 12:00:00  14.65  14.77  14.63  14.77       1  3067300
 2020-12-03 13:00:00  14.76  14.86  14.76  14.83       1  3864800
 2020-12-03 14:00:00  14.82  14.84  14.65 

### Filtering dfs with data (df is not empty)

In [6]:
dfs_with_data = dict(filter(lambda elem: not elem[1].empty, dfs.items()))

In [7]:
dfs_with_data

{'ABEV3':                       Open   High    Low  Close  Spread   Volume
 Date                                                            
 2020-01-02 07:00:00  18.86  18.96  18.78  18.94       1  1503500
 2020-01-02 08:00:00  18.94  19.16  18.91  19.16       1  3092900
 2020-01-02 09:00:00  19.16  19.22  19.13  19.19       1  1742000
 2020-01-02 10:00:00  19.18  19.25  19.17  19.18       1  1865600
 2020-01-02 11:00:00  19.17  19.20  19.15  19.16       1   991200
 ...                    ...    ...    ...    ...     ...      ...
 2020-12-03 10:00:00  14.64  14.74  14.59  14.66       1  3917200
 2020-12-03 11:00:00  14.64  14.75  14.62  14.65       1  3184300
 2020-12-03 12:00:00  14.65  14.77  14.63  14.77       1  3067300
 2020-12-03 13:00:00  14.76  14.86  14.76  14.83       1  3864800
 2020-12-03 14:00:00  14.82  14.84  14.65  14.70       1  5083600
 
 [1694 rows x 6 columns],
 'ACNB34':                         Open     High      Low    Close  Spread  Volume
 Date                 